In [17]:
class FanslyTimestampGenerator {
    /**
     * @param {number} offsetMs - équivalent de this.xdO_ (offset ms à ajouter au timestamp)
     * @param {string|null} sessionId - id de session (optionnel). Si null -> pas d'obfuscation liée à la session.
     */
    constructor(offsetMs = 0, sessionId = null) {
        this.xdO_ = offsetMs;        // offset serveur <-> client
        this.sessionId = sessionId;  // équivalent à getActiveSession().id
        this.cachedTimestamp_ = 0;   // pour garantir la monotonie
    }

    /**
     * Met à jour l'offset (par ex. après avoir recalculé la différence de temps serveur/client).
     */
    setOffsetMs(offsetMs) {
        this.xdO_ = offsetMs;
    }

    /**
     * Met à jour l'id de session (string). Passer null pour "aucune session".
     */
    setSessionId(sessionId) {
        this.sessionId = sessionId;
    }

    /**
     * Mimique le this.getActiveSession() du code original.
     */
    getActiveSession() {
        if (!this.sessionId) return null;
        return { id: this.sessionId };
    }

    /**
     * Reproduction fidèle de getNowTs(e = 0)
     * Retourne un nombre (timestamp "fansly-client-ts").
     */
    getNowTs(e = 0) {
        // Base timestamp + offset
        let r = ((e || Date.now()) + this.xdO_).toString();
        const a = this.getActiveSession();

        if (a) {
            const p = a.id;          // session id (string)
            let g = r.slice(0, -4);  // on enlève les 4 derniers digits du timestamp

            // --- Calcul de h ---
            // h = (avant-dernier digit de p) XOR (dernier digit de g)
            let h = (parseInt(p[p.length - 2], 10) ^ parseInt(g[g.length - 1], 10)).toString();
            if (h.length < 2) {
                h += p[p.length - 1];
            }

            // --- Calcul de v ---
            // v = (dernier digit de p) XOR (avant-dernier digit de g)
            let v = (parseInt(p[p.length - 1], 10) ^ parseInt(g[g.length - 2], 10)).toString();
            if (v.length < 2) {
                v += p[p.length - 3];
            }

            // Reconstruction : on remplace les 4 derniers digits par v + h
            g += v;
            g += h;
            r = g;
        }

        const u = parseInt(r, 10);

        // Monotonie : ne jamais retourner un timestamp plus petit que le précédent
        if (this.cachedTimestamp_ > u) {
            return this.cachedTimestamp_;
        } else {
            this.cachedTimestamp_ = u;
            return u;
        }
    }
}

In [ ]:
async function main() {
  // const sessionId = '853382188782067712';
  const sessionId = '853694184941432875';
  const offsetMs = 0;

  const tsGen = new FanslyTimestampGenerator(offsetMs, sessionId);

  let ts = tsGen.getNowTs(Date.now())

  console.log(ts)

  // Set pour stocker les IDs uniques
  const uniqueIds = new Set();

  await fetch("https://apiv3.fansly.com/api/v1/contentdiscovery/media/suggestionsnew?before=0&after=0&tagIds=&limit=100&offset=0&ngsw-bypass=true", {
  "headers": {
    "accept": "application/json, text/plain, */*",
    "accept-language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
    "authorization": "ODUzMzgyMTg4NzgyMDY3NzEyOjE6MjoxNjIwZmE0MDRmOWFlZjNhYzM1MzhiMWUxZDFlMWM",
    "fansly-client-check": "cd486abc3d367",
    "fansly-client-id": "853370306260135936",
    "fansly-client-ts":  ts.toString(),
    "fansly-session-id": "853694184941432875",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"142\", \"Google Chrome\";v=\"142\", \"Not_A Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "cookie": "_gcl_au=1.1.494078888.1762848142; _ga=GA1.1.583605921.1762848143; intercom-device-id-g2ytx5gg=0d58f8a3-8ada-440f-814e-7a8e274a802a; f-v-d=1764026462591; f-v-v=0.46.94; f-d=853370306260135936; fansly-d=853370306260135936; f-s-c=ODUzMzgyMTg4NzgyMDY3NzEyOjE6MTpmZTMyNzMwMGRiODQzOWQyYWFjYzI1YmQ2ZmEyNzg; intercom-session-g2ytx5gg=ZHAxSHZrL0RkNW84ejlSaUVGSUZVN3lqTXJxVkdyOEQ5NmJuc1dueWNwRzUrcUk4a0FKemllendwU1JXMnFVZy83aTE1VW9zbS9YTE9hcjA2WlpDM2ZiWTBKTFVyUjloODhmUW14WXlZNWM9LS1jZ3pVdXN5dkJiMG9aQ1lXN3pPZzRRPT0=--adee766f86d160ced5a4d38f2d1667eaa747b6c9; _ga_BZSVNWD5W8=GS2.1.s1765031154$o21$g1$t1765031165$j49$l0$h1322146813; amp_4fb08e=VqTAbANLzXWOMxQrG8DjYU...1jbq0s508.1jbq0sfqi.36.0.36; fansly-ts-info={\"tso\":-1674,\"sts\":1765031164501,\"cts\":1765031166175}",
    "Referer": "https://fansly.com/"
  },
  "body": null,
  "method": "GET"
}).then(res => res.json()).then((data) => {
  console.log(data.response.mediaOfferSuggestions.length);
  data.response.mediaOfferSuggestions.forEach(item => uniqueIds.add(item.id));
})

  // Afficher le nombre d'éléments uniques à la fin
  console.log('Nombre d\'éléments uniques:', uniqueIds.size);
}

// En Node 18+ ou avec un fetch global :
main().catch(console.error);


1765195217865


Promise { <pending> }

100
Nombre d'éléments uniques: 100
